### Descarga de capas de mapa de cobertura 4G del Instituto Federal de Telecomunicaciones

In [1]:
import requests
import geopandas as gpd
import pandas as pd
from bs4 import BeautifulSoup
import os
import json

In [2]:
#Crear directorio para guardar los archivos
os.makedirs("geopaquetes", exist_ok=True)
#Cambiar al directorio
os.chdir("geopaquetes")

Paso 1: Se obtiene el código html de la página en donde se encuentra el mapa.

In [3]:
url="https://felt.com/map/Mapa-interactivo-de-cobertura-4G-IFT-nkwEcoI4S9BSa7vgFt6uctD?loc=20.0624,-98.7601,11.6z"

In [4]:
#Obtener el mapa
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# obtener div id="felt-data"
div = soup.find("div", {"id": "felt-data"})
#Eliminar textos
div = str(div).replace('<div id="felt-data" style="display: none">', '')
div = div.replace('\n  </div>', '')
div=div.replace('\n','')

In [6]:
#Transformar a json
data = json.loads(div)

In [7]:
data.keys()

dict_keys(['mapProject', 'teams', 'canCreateMapsSomewhere', 'satelliteMode', 'settings', 'sources', 'sharing', 'mapColorPalette', 'currentTeamBannerAcknowledgement', 'mapbox_api_token', 'felt_version', 'socketTokenTTLSeconds', 'onboardingNeeded', 'customViewport', 'featureFlags', 'isVirtualKarta', 'customIcons', 'flashMessages', 'workspaceBilling', 'mapBackgrounds', 'shareUrl', 'mapTitle', 'showBasemapLabels', 'constraints', 'isTrainingKarta', 'isOwner', 'folderTree', 'defaultZoom', 'mapLinks', 'maptiler_key', 'pipeline', 'elements', 'comments', 'layerGroups', 'actions', 'resyncTimeoutMs', 'mapTeamId', 'maxTileURLLength', 'selectedDefaultBackgroundMode', 'loadedAt', 'urls', 'checkerboardTiles', 'mapId', 'isAdmin', 'layerProcessingEmailSubscriptions', 'isAdminView', 'mapUrls', 'folderId', 'mapDescription', 'validGeoDataExtensions', 'createMapParams', 'editableByCurrentSession', 'mapImages', 'defaultCoordinates', 'maxUserContentFileSizeBytes', 'layerAttrConstraints', 'embedConfig', 'kart

In [8]:
data["layerGroups"][0].keys()

dict_keys(['id', 'name', 'visible', 'description', 'created_at', 'layers', 'created_by', 'modified_at', 'user_id', 'max_zoom', 'index_json_url', 'subtitle', 'z_order', 'hideFromLegend', 'isCollapsed', 'errorMessage', 'thumbnailUrl', 'progress_percent', 'visibilityInteraction', 'created_at_unix_time_ms', 'duplicatedFromId', 'errorType', 'published_to_project_ids', 'renderAsLayer'])

Paso 2: Se obtienen los nombres y urls de la información de cada capa del mapa

In [9]:
#Obtener nombres y urls
nombres=[]
urls=[]
for i in range(len(data["layerGroups"])):
    nombres.append(data["layerGroups"][i]["layers"][0]["normalized"]["layername"])
    urls.append(data["layerGroups"][i]["layers"][0]["index_json_url"])

In [10]:
#Crear tabla
tabla = pd.DataFrame({"nombre":nombres, "url":urls})
#Eliminar último elemento
tabla = tabla[:-1]
#Renombrar último registro como "Movistar"
tabla["nombre"][3]="Movistar"
tabla["nombre"][0]="ATT"
tabla

C:\Users\claud\AppData\Local\Temp\ipykernel_4584\3492863423.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  tabla["nombre"][3]="Movistar"
C:\Users\claud\AppData\Local\Temp\ipykernel_4584\3492863423.py:7: FutureWarning: ChainedAssignmentE

,nombre,url
0,ATT,https://us1.data-pipeline.felt.com/upload/64c9...
1,ALTAN Redes,https://us1.data-pipeline.felt.com/upload/ed2d...
2,AT&amp;T,https://us1.data-pipeline.felt.com/upload/fb71...
3,Movistar,https://us1.data-pipeline.felt.com/upload/938e...


Paso 3: Se obtienen las urls de los datos de cada capa y se descargan los archivos

In [11]:
# Obtener las URLs de los datos de los mapas
urls_data = []
gdf = gpd.GeoDataFrame()

for i in range(len(tabla)):
    url = tabla["url"][i]
    response = requests.get(url)
    data = response.json()

    # Verificar si 'data_url' es válido
    data_url = data["datasets"][0].get("data_url")
    if not data_url:  # Si es None o vacío, saltar al siguiente
        print(f"URL inválida o faltante en la capa {tabla['nombre'][i]}, saltando...")
        continue

    urls_data.append(data_url)

    # Descargar el geopackage con el nombre de la capa
    nombre_archivo = tabla["nombre"][i]
    write_path = f"{nombre_archivo}.gpkg"
    with open(write_path, "wb") as f:
        f.write(requests.get(data_url).content)
        print(f"Descargando {nombre_archivo}.gpkg...")
        print(f"Archivo guardado en {write_path}")


URL inválida o faltante en la capa ATT, saltando...
Descargando ALTAN Redes.gpkg...
Archivo guardado en ALTAN Redes.gpkg
Descargando AT&amp;T.gpkg...
Archivo guardado en AT&amp;T.gpkg
Descargando Movistar.gpkg...
Archivo guardado en Movistar.gpkg


Paso 4: Se cargan los archivos geopackage y se concatenan en un solo archivo

In [12]:
gdfs = []
#Cargar datos
for i in range(len(tabla)):
    nombre_archivo = tabla['nombre'][i]
    try:
        gdf = gpd.read_file(f"{nombre_archivo}.gpkg")
        #Incluir el nombre de la capa
        gdf["nombre"] = tabla["nombre"][i]
        gdfs.append(gdf)
        print(f"Archivo {nombre_archivo}.gpkg cargado correctamente")
    except:
        print(f"Error al cargar el archivo {nombre_archivo}.gpkg")
        
# Concatenar los geodataframes
gdf_final = pd.concat(gdfs, ignore_index=True)

Error al cargar el archivo ATT.gpkg
Archivo ALTAN Redes.gpkg cargado correctamente
Archivo AT&amp;T.gpkg cargado correctamente
Archivo Movistar.gpkg cargado correctamente


In [13]:
gdf_final

,felt:feature,felt:has_geometry,felt:h3_index,Calidad,geometry,nombre
0,1,True,None,Excelente,"MULTIPOLYGON (((-117.12184 32.52598, -117.1241...",ALTAN Redes
1,2,True,None,Buena,"MULTIPOLYGON (((-117.12165 32.50254, -117.1239...",ALTAN Redes
2,3,True,None,Buena,"MULTIPOLYGON (((-117.12162 32.49919, -117.1239...",ALTAN Redes
3,4,True,None,Excelente,"MULTIPOLYGON (((-116.89474 32.50399, -116.8970...",ALTAN Redes
4,5,True,None,Excelente,"MULTIPOLYGON (((-116.89819 32.50559, -116.9004...",ALTAN Redes
...,...,...,...,...,...,...
1000191,516904,True,None,Buena,"MULTIPOLYGON (((-99.15503 20.46778, -99.15709 ...",Movistar
1000192,516905,True,None,Buena,"MULTIPOLYGON (((-104.70523 19.26123, -104.7073...",Movistar
1000193,516906,True,None,Regular,"MULTIPOLYGON (((-110.87966 28.97732, -110.8819...",Movistar
1000194,516907,True,None,Regular,"MULTIPOLYGON (((-104.83579 27.22782, -104.8380...",Movistar


In [14]:
type(gdf_final)

geopandas.geodataframe.GeoDataFrame

Paso 5: Se guarda el archivo final en formato geopackage

In [15]:
#Salvar como geopackage
gdf_final.to_file("cobertura_4g.gpkg", driver="GPKG",crs="EPSG:4326")